### Question: does `model` returns `[0 prob, 1 prob]` before softmax?

In [1]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    TextClassificationPipeline,
)
from engine.data import prepare_data_for_fine_tuning, read_data
from engine.xai import FeatureAblationText

2024-11-18 15:37:50.864733: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-18 15:37:50.889209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 15:37:50.889232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 15:37:50.890023: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 15:37:50.895371: I tensorflow/core/platform/cpu_feature_guar

In [2]:
MODEL_ID = "roberta-base"
MODEL_PATH = "output/checkpoint-2025/model.safetensors"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [3]:
test = read_data("data/split_raw/test.tsv")
test_dataset = prepare_data_for_fine_tuning(test, tokenizer)

Map:   0%|          | 0/790 [00:00<?, ? examples/s]

In [4]:
config = AutoConfig.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH, config=config
)
model.eval()
pipeline = TextClassificationPipeline(
    model=model, tokenizer=tokenizer, top_k=2, device="cuda"
)

In [5]:
%%capture
model.cuda()

In [6]:
obs = test["text"].tolist()[0]
obs_pt = pipeline.tokenizer(obs, return_tensors="pt")['input_ids'].cuda()

In [7]:
def forward(obs):
    return model(obs).logits

In [8]:
attr = FeatureAblationText(forward)
exp = attr.get_attributions([obs_pt])
exp

tensor([[[ 0.0000e+00,  1.2517e-06,  3.8296e-06,  2.4587e-06,  6.4671e-06,
           1.7881e-06,  4.5002e-06,  3.7551e-06,  8.7023e-06,  4.3213e-06,
           1.2949e-05,  4.6045e-06,  4.6343e-06,  1.8775e-06,  5.5134e-07,
           5.3793e-06,  4.9323e-06,  6.5416e-06],
         [ 0.0000e+00,  1.3225e-07,  1.9427e-06,  1.2070e-06,  2.5798e-06,
           9.7975e-07,  2.2072e-06,  1.8626e-06,  4.2524e-06,  2.2631e-06,
           5.9139e-06,  1.8496e-06,  1.8291e-06,  8.1584e-07, -1.0617e-07,
           2.6729e-06,  2.9225e-06,  3.6415e-06]]], device='cuda:0')

In [9]:
pipeline.tokenizer.convert_ids_to_tokens(obs_pt[0])

['<s>',
 'Building',
 'Ġa',
 'Ġwall',
 'Ġon',
 'Ġthe',
 'ĠU',
 '.',
 'S',
 '.-',
 'Mexico',
 'Ġborder',
 'Ġwill',
 'Ġtake',
 'Ġliterally',
 'Ġyears',
 '.',
 '</s>']